# Potato Disease Classification
## Machine Learning Pipeline - Summative Project

**Author:** Edine Noella Mugisha 

**Date:** November 2024  
**Objective:** Build an end-to-end ML pipeline for classifying potato leaf diseases

---

### Problem Statement
Small-scale farmers in Africa lose 30-50% of their potato crop yield to diseases annually. The two main diseases affecting potatoes are:

1. **Early Blight** (Alternaria solani)
2. **Late Blight** (Phytophthora infestans)

### Solution
Build an AI-powered image classification system to:
- Detect diseases early from leaf images
- Provide instant diagnosis with confidence scores
- Offer treatment recommendations
- Enable accessible expert diagnosis

### Dataset
- **Source:** PlantVillage Dataset (https://github.com/spMohanty/PlantVillage-Dataset)
- **Classes:** 3 (Early Blight, Late Blight, Healthy)
- **Total Images:** 2,152
- **Format:** JPG, RGB

In [1]:
# Import Libraries
#-------------------

# Core Libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Image Processing
from PIL import Image
import cv2

# Data Download
import urllib.request
import zipfile

# Machine Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

# Utilities
import json
import time
from datetime import datetime

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

# Check TensorFlow version
print(f"TensorFlow Version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

TensorFlow Version: 2.20.0
GPU Available: []


In [ ]:
# Data Acquisition
#-----------------

import os

print("\n" + "="*80)
print("DATA ACQUISITION")
print("="*80)

# Root of your project (optional: you can hardcode if you prefer)
project_dir = '/Users/edine.mugisha/Documents/School/potato_disease_summative_MLOP'

# Data folder
data_root = os.path.join(project_dir, 'data')

# Dataset folder (this is the folder you manually put there)
dataset_dir = os.path.join(data_root, 'PlantVillage-Dataset-master')

# Check that the dataset folder exists
if not os.path.exists(dataset_dir):
    raise FileNotFoundError(
        f"Dataset folder not found at:\n  {dataset_dir}\n"
        "Make sure you downloaded and unzipped the PlantVillage repo into the 'data' folder."
    )

print(f"✓ Dataset folder found at: {dataset_dir}")

# Base path where class folders live
base_path = os.path.join(dataset_dir, 'raw', 'color')

if not os.path.exists(base_path):
    raise FileNotFoundError(
        f"'raw/color' folder not found inside dataset folder:\n  {base_path}\n"
        "Check that the folder structure matches: PlantVillage-Dataset-master/raw/color/..."
    )

# List all available categories
all_categories = sorted([
    d for d in os.listdir(base_path)
    if os.path.isdir(os.path.join(base_path, d))
])

print(f"\n✓ Found {len(all_categories)} categories:")
for c in all_categories:
    print("  -", c)



DATA ACQUISITION


KeyboardInterrupt: 

In [ ]:
# Select potato-related categories
#---------------------------------

print("\n" + "="*80)
print("SELECTING POTATO DISEASE CLASSES")
print("="*80)

# Focus on potato diseases
SELECTED_CLASSES = [
    'Potato___Early_blight',
    'Potato___Late_blight',
    'Potato___healthy'
]

categories = [cat for cat in all_categories if cat in SELECTED_CLASSES]

print(f"\n✓ Selected {len(categories)} classes:")
for i, cat in enumerate(categories, 1):
    count = len(os.listdir(os.path.join(base_path, cat)))
    print(f"  {i}. {cat}: {count} images")


In [ ]:
# EDA - Class Distribution
#-------------------------

print("\n" + "="*80)
print("EXPLORATORY DATA ANALYSIS")
print("="*80)

# Count images per class
class_counts = {}
for category in categories:
    class_path = os.path.join(base_path, category)
    class_counts[category] = len(os.listdir(class_path))

# Create DataFrame
df_counts = pd.DataFrame(list(class_counts.items()), 
                         columns=['Class', 'Count'])
df_counts['Class'] = df_counts['Class'].str.replace('Potato___', '').str.replace('_', ' ')

print("\nClass Distribution:")
print(df_counts)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Bar plot
axes[0].bar(df_counts['Class'], df_counts['Count'], 
           color=['#e74c3c', '#e67e22', '#27ae60'])
axes[0].set_title('Class Distribution', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Disease Class')
axes[0].set_ylabel('Number of Images')
axes[0].tick_params(axis='x', rotation=45)

# Pie chart
colors = ['#e74c3c', '#e67e22', '#27ae60']
axes[1].pie(df_counts['Count'], labels=df_counts['Class'], 
           autopct='%1.1f%%', colors=colors, startangle=90)
axes[1].set_title('Class Distribution (%)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig('eda_class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✓ Total images: {df_counts['Count'].sum()}")

In [ ]:
# EDA - Sample Images
#--------------------

# Display sample images from each class
fig, axes = plt.subplots(3, 4, figsize=(16, 12))

for i, category in enumerate(categories):
    class_path = os.path.join(base_path, category)
    images = os.listdir(class_path)[:4]
    
    for j, img_name in enumerate(images):
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path)
        
        axes[i, j].imshow(img)
        axes[i, j].axis('off')
        if j == 0:
            clean_name = category.replace('Potato___', '').replace('_', ' ')
            axes[i, j].set_title(clean_name, fontsize=12, fontweight='bold')

plt.suptitle('Sample Images from Each Class', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('eda_sample_images.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Image Properties Analysis
#-------------------------

# Analyze image properties
image_sizes = []
aspect_ratios = []

for category in categories:
    class_path = os.path.join(base_path, category)
    images = os.listdir(class_path)[:50]  # Sample 50 images per class
    
    for img_name in images:
        img_path = os.path.join(class_path, img_name)
        img = Image.open(img_path)
        width, height = img.size
        image_sizes.append((width, height))
        aspect_ratios.append(width / height)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Image sizes
widths = [s[0] for s in image_sizes]
heights = [s[1] for s in image_sizes]
axes[0].scatter(widths, heights, alpha=0.5)
axes[0].set_title('Image Dimensions Distribution', fontweight='bold')
axes[0].set_xlabel('Width (pixels)')
axes[0].set_ylabel('Height (pixels)')
axes[0].grid(True, alpha=0.3)

# Aspect ratios
axes[1].hist(aspect_ratios, bins=30, color='steelblue', alpha=0.7)
axes[1].set_title('Aspect Ratio Distribution', fontweight='bold')
axes[1].set_xlabel('Aspect Ratio')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('eda_image_properties.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Average Width: {np.mean(widths):.0f} pixels")
print(f"Average Height: {np.mean(heights):.0f} pixels")
print(f"Average Aspect Ratio: {np.mean(aspect_ratios):.2f}")

In [ ]:
# Data Preprocessing Setup
#-------------------------

print("\n" + "="*80)
print("DATA PREPROCESSING")
print("="*80)

# Configuration
IMG_SIZE = (256, 256)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2

print(f"\nConfiguration:")
print(f"  Image Size: {IMG_SIZE}")
print(f"  Batch Size: {BATCH_SIZE}")
print(f"  Validation Split: {VALIDATION_SPLIT}")

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest',
    validation_split=VALIDATION_SPLIT
)

# Only rescaling for validation
val_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=VALIDATION_SPLIT
)

print("\n✓ Data augmentation configured")

In [ ]:
# Load Data
#-----------

# Create generators
train_generator = train_datagen.flow_from_directory(
    base_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    classes=SELECTED_CLASSES,
    shuffle=True
)

val_generator = val_datagen.flow_from_directory(
    base_path,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    classes=SELECTED_CLASSES,
    shuffle=False
)

# Get class information
class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

print(f"\n✓ Training samples: {train_generator.samples}")
print(f"✓ Validation samples: {val_generator.samples}")
print(f"✓ Number of classes: {len(class_names)}")
print(f"✓ Class names: {class_names}")

# Save class names
with open('class_names.json', 'w') as f:
    json.dump(class_names, f)

In [ ]:
# Visualize Augmentation
#-----------------------

# Visualize data augmentation
def visualize_augmentation(generator, num_samples=9):
    # Get a batch
    images, labels = next(generator)
    
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    axes = axes.ravel()
    
    for i in range(num_samples):
        axes[i].imshow(images[i])
        class_idx = np.argmax(labels[i])
        class_name = class_names[class_idx].replace('Potato___', '').replace('_', ' ')
        axes[i].set_title(f'{class_name}', fontweight='bold')
        axes[i].axis('off')
    
    plt.suptitle('Augmented Training Images', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig('augmented_samples.png', dpi=300, bbox_inches='tight')
    plt.show()

visualize_augmentation(train_generator)

In [ ]:
# Model Building (Custom CNN + Transfer Learning)
#-----------------------------------------------

print("\n" + "="*80)
print("MODEL BUILDING")
print("="*80)

from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Conv2D, MaxPooling2D, Flatten, Dense, Dropout, 
                                     BatchNormalization, GlobalAveragePooling2D)
from tensorflow.keras.optimizers import Adam

input_shape = IMG_SIZE + (3,)
num_classes = len(class_names)

def build_cnn_model(input_shape, num_classes):
    """
    Simple baseline CNN built from scratch.
    """
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),

        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    return model


def build_transfer_model(input_shape, num_classes):
    """
    Transfer Learning using MobileNetV2 as feature extractor.
    """
    base_model = MobileNetV2(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    
    # Freeze base model for initial training
    base_model.trainable = False
    
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=base_model.input, outputs=outputs)
    return model

# Choose which model to use: "cnn" or "transfer"
MODEL_TYPE = "transfer"   # <- change to "cnn" if you want the scratch CNN

if MODEL_TYPE == "cnn":
    model = build_cnn_model(input_shape, num_classes)
    model_name = "potato_cnn"
else:
    model = build_transfer_model(input_shape, num_classes)
    model_name = "potato_mobilenetv2"

# Compile model
learning_rate = 1e-4
model.compile(
    optimizer=Adam(learning_rate=learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"\n✓ Model type: {MODEL_TYPE}")
print(f"✓ Model name: {model_name}")
model.summary()


In [ ]:
# Training Setup with Callbacks
#--------------------------------

print("\n" + "="*80)
print("TRAINING SETUP")
print("="*80)

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Make directory for models
os.makedirs("models", exist_ok=True)

checkpoint_path = os.path.join("models", f"{model_name}_best.h5")

checkpoint_cb = ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    save_best_only=True,
    mode='max',
    verbose=1
)

earlystop_cb = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_cb = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

callbacks = [checkpoint_cb, earlystop_cb, reduce_lr_cb]

EPOCHS = 20  # you can increase to e.g. 30 if training is stable

steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = val_generator.samples // BATCH_SIZE

print(f"\nTraining configuration:")
print(f"  Epochs: {EPOCHS}")
print(f"  Steps per epoch: {steps_per_epoch}")
print(f"  Validation steps: {validation_steps}")
print(f"  Checkpoint path: {checkpoint_path}")


In [ ]:
# Model Training
#---------------

print("\n" + "="*80)
print("MODEL TRAINING")
print("="*80)

start_time = time.time()

history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    epochs=EPOCHS,
    callbacks=callbacks
)

end_time = time.time()
training_time = end_time - start_time

print(f"\n✓ Training completed in {training_time/60:.2f} minutes")


In [ ]:
# Training History Visualization
#--------------------------------

print("\n" + "="*80)
print("TRAINING HISTORY VISUALIZATION")
print("="*80)

# Convert history to DataFrame
history_df = pd.DataFrame(history.history)
print("\nHistory keys:", history_df.columns.tolist())

# Plot Accuracy
plt.figure(figsize=(10, 6))
plt.plot(history_df['accuracy'], label='Training Accuracy')
plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy over Epochs', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('training_accuracy_curve.png', dpi=300, bbox_inches='tight')
plt.show()

# Plot Loss
plt.figure(figsize=(10, 6))
plt.plot(history_df['loss'], label='Training Loss')
plt.plot(history_df['val_loss'], label='Validation Loss')
plt.title('Model Loss over Epochs', fontweight='bold')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('training_loss_curve.png', dpi=300, bbox_inches='tight')
plt.show()

# Save history to JSON (for reproducibility)
with open(f'{model_name}_history.json', 'w') as f:
    json.dump(history.history, f)

print("\n✓ Training curves saved as PNG")
print("✓ History saved as JSON")


In [ ]:
# Model Evaluation
#-----------------

print("\n" + "="*80)
print("MODEL EVALUATION ON VALIDATION SET")
print("="*80)

# Ensure we evaluate with the best weights (best checkpoint already restored by EarlyStopping)
val_loss, val_acc = model.evaluate(val_generator, verbose=1)
print(f"\nValidation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")


In [ ]:
# Classification Report and Accuracy
#-----------------------------------

print("\n" + "="*80)
print("DETAILED EVALUATION METRICS")
print("="*80)

# Reset generator to start from beginning
val_generator.reset()

# Get predictions (probabilities)
y_prob = model.predict(val_generator, verbose=1)
y_pred = np.argmax(y_prob, axis=1)

# True labels
y_true = val_generator.classes
target_names = list(val_generator.class_indices.keys())

# Overall accuracy
acc = accuracy_score(y_true, y_pred)
print(f"\nOverall Accuracy: {acc:.4f}\n")

# Classification report
report = classification_report(y_true, y_pred, target_names=target_names)
print("Classification Report:")
print(report)


In [ ]:
# Confusion Matrix
#------------------

print("\n" + "="*80)
print("CONFUSION MATRIX")
print("="*80)

cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=target_names,
            yticklabels=target_names)
plt.title('Confusion Matrix', fontweight='bold')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# ROC Curves (One-vs-Rest)
#------------------------

print("\n" + "="*80)
print("ROC CURVES (ONE-vs-REST)")
print("="*80)

from sklearn.metrics import roc_curve, auc
from tensorflow.keras.utils import to_categorical

# Binarize labels
y_true_bin = to_categorical(y_true, num_classes=num_classes)

fpr = {}
tpr = {}
roc_auc = {}

for i in range(num_classes):
    fpr[i], tpr[i], _ = roc_curve(y_true_bin[:, i], y_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Micro-average ROC
fpr["micro"], tpr["micro"], _ = roc_curve(y_true_bin.ravel(), y_prob.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

plt.figure(figsize=(10, 8))

for i, class_name in enumerate(target_names):
    plt.plot(fpr[i], tpr[i], label=f'{class_name} (AUC = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
plt.title('ROC Curves (One-vs-Rest)', fontweight='bold')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('roc_curves.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nAUC scores per class:")
for i, class_name in enumerate(target_names):
    print(f"  {class_name}: {roc_auc[i]:.3f}")


In [ ]:
# Sample Predictions
#-------------------

print("\n" + "="*80)
print("SAMPLE PREDICTIONS")
print("="*80)

# Get one batch of validation images
val_batch_images, val_batch_labels = next(val_generator)

# Get predictions for this batch
batch_prob = model.predict(val_batch_images)
batch_pred = np.argmax(batch_prob, axis=1)
batch_true = np.argmax(val_batch_labels, axis=1)

num_samples = 9
plt.figure(figsize=(14, 14))

for i in range(num_samples):
    plt.subplot(3, 3, i + 1)
    img = val_batch_images[i]
    true_label = target_names[batch_true[i]]
    pred_label = target_names[batch_pred[i]]
    confidence = np.max(batch_prob[i]) * 100

    plt.imshow(img)
    plt.axis('off')
    
    title_color = 'green' if true_label == pred_label else 'red'
    plt.title(
        f"True: {true_label.replace('Potato___', '').replace('_', ' ')}\n"
        f"Pred: {pred_label.replace('Potato___', '').replace('_', ' ')} ({confidence:.1f}%)",
        color=title_color,
        fontsize=10
    )

plt.suptitle('Sample Validation Predictions', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('sample_predictions.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Save Final Model and Artifacts
#-------------------------------

print("\n" + "="*80)
print("SAVING MODEL & ARTIFACTS")
print("="*80)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
export_dir = os.path.join("models", f"{model_name}_{timestamp}")
os.makedirs(export_dir, exist_ok=True)

# 1. Save Keras .h5 model
h5_path = os.path.join(export_dir, f"{model_name}.h5")
model.save(h5_path)
print(f"✓ Model saved as H5 at: {h5_path}")

# 2. Save TensorFlow SavedModel format
savedmodel_path = os.path.join(export_dir, "saved_model")
model.save(savedmodel_path)
print(f"✓ Model saved in SavedModel format at: {savedmodel_path}")

# 3. Save class names (again, with timestamped path)
class_names_path = os.path.join(export_dir, "class_names.json")
with open(class_names_path, 'w') as f:
    json.dump(class_names, f)
print(f"✓ Class names saved at: {class_names_path}")

# 4. Save latest history as JSON
history_path = os.path.join(export_dir, "training_history.json")
with open(history_path, 'w') as f:
    json.dump(history.history, f)
print(f"✓ Training history saved at: {history_path}")

print("\nAll artifacts saved successfully.")
